In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent  # or Path().resolve().parent
sys.path.insert(0, str(project_root))

from src.evaluate import evaluate

In [ ]:
import json


# Load the base model data and extract the outputs 
with open("../data/base_outputs.json", "r", encoding="utf-8") as f:
    model0_data = json.load(f)

model0_outputs = []
for i in model0_data:
    model0_outputs.append(i[1])

#load the ground truth / reference data and unpack the outputs
from datasets import Dataset
ref_data = []
with open("../data/custom_dataset.jsonl", "r") as f:
    for line in f:
        ref_data.append(json.loads(line))

custom_dataset = Dataset.from_dict({
    "messages": [item["messages"] for item in ref_data]
})
split_dataset = custom_dataset.train_test_split(test_size=0.1, seed=42)
test_dataset = split_dataset["test"]

print(f"✓ Loaded {len(test_dataset)} examples")

reference = []
for i in test_dataset:
    reference.append(i["messages"][2]["content"])



✓ Loaded 101 examples


In [ ]:
p, r, zeroF1 = evaluate(model0_outputs, reference, device = "mps")
# p, r, oneF1 = evaluate(generations, reference, device = "mps")
# p, r, twoF1 = evaluate(generations, grounds, device = "mps")


In [ ]:
import numpy as np
from scipy.stats import tukey_hsd

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1)
ax.boxplot([group0, group1, group2])
ax.set_xticklabels(["group0", "group1", "group2"]) 
ax.set_ylabel("mean") 
plt.show()

In [ ]:
res = tukey_hsd(group0, group1, group2)
print(res)